# Numerical Methods for Finding Polynomial Roots
This notebook is dedicated to algorithms useful for finding the roots of a given polynomial.  
Here you'll find algorithms for **Bisection** and **Newton-Raphson**.

# Disclaimer

**Important:**  
For the correct execution of this notebook, please make sure to run the cells sequentially (top to bottom).  
Some cells depend on the output of previous ones, and running them out of order may result in errors or incorrect results.

Start from the first cell and work your way down for the best experience.



## 📐 Bisection Method

The Bisection method uses a simple algorithm that allows you to find approximate solutions of functions within a given interval.  
It is important to note that this algorithm **cannot** find all roots of a polynomial. Instead, we need to input an interval where a root could exist.  
Given a known interval `[a, b]` where the function is continuous and it's known that a root exists, the approximation is given by:

$$ c = \frac{a + b}{2} $$

This interval must satisfy **Bolzano's Theorem** to ensure the function crosses the x-axis.

### Bolzano's Theorem Formula
If $ f(a) \cdot f(b) < 0 $, then there exists a root $x_i \in (a, b) $ such that $ f(x_i) = 0 $.


In [2]:
#importing Libraries
import numpy as np
import plotly.graph_objects as go
import pandas as pd

In [3]:
#Creating the function to be used in this notebook

def fun(x):
    return np.exp( - x ** 3) - (2 * x) + 1

In [4]:
#Bisection Method

def bisection(function, a, b, c, tolerance= 0.01, max_interations=50):
    """
    This function is expected to return a list of dictionaries containing the values 
    of a, b, c, f(a), f(b), f(c) and the last value of c."""

    #Test Bolzano theorem
    if function(a) * function(c) > 0:
        print("Bolzano theorem is not satisfied")
        return [], None
    
    #Start iterating the algorithm
    iter_data = []
    c_prev = 2 * c #For display purposes only in order to make the first iteration 100% error
    for i in range(max_interations):

        c = (a + b) / 2
        calculated_error = round(abs(c - c_prev) / c, 4)

        iter_data.append({
            'iteration': i,
            'a': a,
            'b': b,
            'c': c,
            'f(a)': function(a),
            'f(b)': function(b),
            'f(c)': function(c),
            '%error': calculated_error * 100,
            })

        #Stop condition
        if calculated_error < tolerance:
            break
        #Update a and b
        if function(c) * function(a) < 0:
            b = c
        else:
            a = c
        c_prev = c
    return iter_data, c

#initial values

a_initial = 0.75
b_initial = 1
c_initial = (a_initial + b_initial) / 2
max_interations = 10

#creating the istances of the bisection function
iterations_bisection, root = bisection(fun, a_initial, b_initial, c_initial, max_interations=max_interations)

#Creating the values for the plot
x_values = np.linspace(a_initial - 0.5, b_initial + 0.5, 150)
y_values = fun(x_values)


#Plotting the function and the iterations
fig = go.Figure(
    data = [
        #Plotting the function
        go.Scatter(x = x_values, y = y_values, mode = 'lines', name = 'f(x)', 
                  line = dict(color = 'blue', width = 2.5)),
        #Plotting the initial values
        go.Scatter(x= [a_initial, b_initial], y = [fun(a_initial), fun(b_initial)], mode = 'markers',
                  name = '[a, b]', marker = dict(color = 'red', size = 10)),
        #Plotting the initial value of c
        go.Scatter(x = [c_initial], y = [fun(c_initial)], mode = 'markers', name = 'c Value',
                  marker = dict(color = 'green', size = 10)),
    ],
    layout=go.Layout(
        xaxis = dict(range=[min(x_values), max(x_values)], title='x'),
        yaxis = dict(range=[min(y_values), max(y_values)], title='f(x)'),
        title = 'Bisection Method (Iteration 0)',
        updatemenus=[dict(
            type='buttons',
            buttons=[dict(label='Play',
                          method='animate',
                          args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "transition": {"duration": 300, "easing": "quadratic-in-out"}}])]
        )],
    
    )
)


#Creating the frames for the animation
frames = []
#Iterating the frames that compose the animation
for i, data in enumerate(iterations_bisection):
    frame_title = f"Bisection Method (Iteration {data['iteration']})"
    frame = go.Frame(
        data=[
            go.Scatter(x=x_values, y=y_values, mode='lines', name='f(x)',
                       line=dict(color='blue', width=2.5)),
            go.Scatter(x=[data['a'], data['b']], y=[data['f(a)'], data['f(b)']],
                       mode='markers', name='[a, b]', marker=dict(size=10, color='red')),
            go.Scatter(x=[data['c']], y=[data['f(c)']],
                       mode='markers', name='c Value', marker=dict(size=10, color='green', symbol='circle'))
        ],
        name=f'iteration_{i}',
        layout=go.Layout(title_text=frame_title)
    )
    frames.append(frame)

fig.frames = frames

# Add slider to control the animation
sliders = [dict(
    steps=[dict(method='animate',
                args=[[f'iteration_{k}'],
                      dict(mode='immediate',
                           frame=dict(duration=500, redraw=True),
                           transition=dict(duration=300))],
                label=str(k))
           for k in range(len(fig.frames))],
    transition=dict(duration=300),
    x=0.08,
    len=0.9,
    currentvalue=dict(font=dict(size=12), prefix="Iteration: ", visible=True, xanchor="right"),
    pad=dict(b=10, t=50)
)]

fig.update_layout(sliders=sliders)


fig.show()



In [5]:
#Plotting a table with the iterations

"""We will use the list of dictionaries created in the previous code to create the table
please note that it is necessary to execute the code above first to to display the table"""

#Creating the dataframe for the bisection method
df_bisection = pd.DataFrame(iterations_bisection)

#Show table
df_bisection.head(max_interations)


,iteration,a,b,c,f(a),f(b),f(c),%error
0,0,0.750000,1.0000,0.875000,0.155816,-0.632121,-0.238251,100.00
1,1,0.750000,0.8750,0.812500,0.155816,-0.238251,-0.040137,7.69
2,2,0.750000,0.8125,0.781250,0.155816,-0.040137,0.058244,4.00
3,3,0.781250,0.8125,0.796875,0.058244,-0.040137,0.009138,1.96
4,4,0.796875,0.8125,0.804688,0.009138,-0.040137,-0.015480,0.97


## ⚙️ Newton-Raphson Method

The Newton-Raphson algorithm is a more refined one, capable of obtaining more precise approximations in fewer iterations.  
It is important to consider certain stopping conditions for the iteration algorithm:

1. If the **limit** of the function evaluated at the current value of $ x $ tends to 0, stop.
3. If the error value falls below the set tolerance, stop.

### Newton-Raphson Iteration Algorithm

The iteration algorithm is given by:

$$ x_{j+1} = x_j - \frac{f(x_j)}{f'(x_j)} $$


In [11]:
#Newthon-Rhapson Method for root finding
def derivative(x):
    return - (3 * x ** 2 * np.exp(- x ** 3)) - 2
def newton_raphson(function, derivative, xi, tolerance=0.0001, maximum_iterations=50):
    #setting a limit
    limit = 0.0000001
    #creating a list for storing the iterations data
    iterations_newton = []
    iterations_newton.append({
        'iteration': 0,
        'xi': xi,
        'f(xi)': function(xi),
        '%Error': 100,  # Initial error set to 100%
    })
    #Start iterating the algorithm
    for i in range(maximum_iterations):
        #Culculating xi+1
        xi_plus_1 = xi - (function(xi) / derivative(xi))
        #calculating error
        error = abs((xi_plus_1 - xi) / xi_plus_1) * 100
        iterations_newton.append({
            'iteration': i + 1,
            'xi': xi_plus_1,
            'f(xi)': function(xi_plus_1),
            '%Error': round(error,4)
        })
        #Stop conditions
        if error < tolerance or abs(function(xi_plus_1)) < limit:
            break
        #Update xi for the next iteration
        xi = xi_plus_1
    return iterations_newton, xi_plus_1

#Creeating an instance of the newton-raphson method
x_initial = 0.6
iterations_newton, root_newton = newton_raphson(fun, derivative, x_initial)


#Crating linear space for the plot
x_values_newton = np.linspace(x_initial - 0.5, x_initial + 0.5, 150)
y_values_newton = fun(x_values_newton)

#Creating the plot for the newton-raphson method
fig_newton = go.Figure(
    data=[
        #Plotting the function
        go.Scatter(x=x_values_newton, y=y_values_newton, mode='lines', name='f(x)',
                   line=dict(color='blue', width=2)),
        #Plotting the initial value
        go.Scatter(x=[x_initial], y=[fun(x_initial)], mode='markers', name='f(xi)',
                   marker=dict(color='green', size=10)),
    ],
    layout=go.Layout(
        xaxis=dict(range=[min(x_values_newton), max(x_values_newton)], title='x'),
        yaxis=dict(range=[min(y_values_newton), max(y_values_newton)], title='f(x)'),
        title='Newton-Raphson Method',
        updatemenus=[dict(
            type='buttons',
            buttons=[dict(label='Play',
                          method='animate',
                          args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True, "transition": {"duration": 300, "easing": "quadratic-in-out"}}])]
        )],
    )
)

#Creating the frames for the animation
frames_newton = []
for i, data in enumerate(iterations_newton):
    frame_title = f"Newton-Rhapson Method (Iteration {data['iteration']})"
    frame_newton = go.Frame(
        data=[
            go.Scatter(x=x_values_newton, y=y_values_newton, mode='lines', name='f(x)',
                       line=dict(color='blue', width=2)),
            go.Scatter(x=[data['xi']], y=[data['f(xi)']],
                       mode='markers', name='f(xi)', marker=dict(size=10, color='green')),
        ],
        name=f'iteration_{i}',
        layout=go.Layout(title_text=frame_title)
    )
    frames_newton.append(frame_newton)

fig_newton.frames = frames_newton

# Add slider to control the animation

sliders = [dict(
    steps=[dict(method='animate',
                args=[[f'iteration_{k}'],
                      dict(mode='immediate',
                           frame=dict(duration=500, redraw=True),
                           transition=dict(duration=300))],
                label=str(k))
           for k in range(len(fig_newton.frames))],
    transition=dict(duration=300),
    x=0.08,
    len=0.9,
    currentvalue=dict(font=dict(size=12), prefix="Iteration: ", visible=True, xanchor="right"),
    pad=dict(b=10, t=50)
)]

fig_newton.update_layout(sliders=sliders)

fig_newton.show()

Next you can execute the code to see the table of iterations

In [7]:
#Creating the table to show the iterations

df_newton = pd.DataFrame(iterations_newton)
#Show table 
df_newton.head(max_interations)


,iteration,xi,f(xi),%Error
0,0,0.600000,6.057353e-01,100.0000
1,1,0.811043,-3.553656e-02,26.0212
2,2,0.799789,-3.821037e-05,1.4072
3,3,0.799776,-4.922285e-11,0.0015
